In [58]:
import pandas as pd
import numpy as np
import os
from tkinter import Tk, Label, Button, filedialog, Listbox, Text, Scrollbar, END, MULTIPLE
from tkinter.messagebox import showinfo
from collections import Counter, defaultdict

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10000)

menu_font_size = 20
submenu_font_size = 12
last_sorting_method = None

def calc_revenue(df, month=None, store_name=None):
    df.columns = [col.replace('\n', '') for col in df.columns]
    condition = df['狀態'].apply(lambda x: x.split('\n')[0] not in ['取消訂單', '逾期未取件']) 
    df = df.loc[condition]
    
    if month:
        condition = df['訂購日期'].apply(lambda x: x.split('/')[1] in month)
        df = df.loc[condition]

    if store_name:
        condition = df['賣場名稱'].apply(lambda x: x in store_name)
        df = df.loc[condition]

    revenue = df['小計(A)'].apply(lambda x: x.replace(',', '')).astype(int).sum()
    return revenue


def read_data(df_paths):
    df_list = [pd.read_excel(df_path, skiprows=2) for df_path in df_paths]
    return pd.concat(df_list, axis=0)


def select_files():
    file_paths = filedialog.askopenfilenames(title="選擇檔案", filetypes=[("Excel files", "*.xlsx *.xls")])
    if file_paths:
        file_listbox.delete(0, END)
        selected_files.clear()
        selected_files.extend(file_paths)

        for file in file_paths:
            file_listbox.insert(END, os.path.basename(file))


def calculate_revenue():
    if not selected_files:
        showinfo("提示", "未選擇任何檔案")
        return
    
    selected_months = [month_listbox.get(i) for i in month_listbox.curselection()]
    month = [str(month).zfill(2) for month in selected_months]

    store_name = None
    df = read_data(selected_files)
    revenue = calc_revenue(df=df, month=month, store_name=store_name)
    showinfo("營收結果", f"總營收: {revenue}", font=('Arial', 15))


def load_stroke_data(filepath):
    # Load CSV and create a dictionary of character to stroke count
    df = pd.read_csv(filepath, skiprows=4)
    stroke_dict = dict(zip(df['Character'], df['Strokes']))
    return stroke_dict

def stroke_sort(input_list, stroke_dict):
    def char_key(char):
        # If the character is in stroke dictionary, return its stroke count and char for further sorting
        if char in stroke_dict:
            return (stroke_dict[char], char)
        # Non-Chinese characters are placed with a default stroke count of 0 for initial sorting
        else:
            return (0, char)

    def string_key(s):
        # Convert each string into a tuple of sorting keys for each character
        return [char_key(char) for char in s]

    # Sort the list based on generated keys
    sorted_list = sorted(input_list, key=string_key)
    return sorted_list


def calculate_buyer_data(df, sort_by=None):
    df_filtered = df.loc[df.iloc[:, 0].map(lambda x: x != '下架日/開售日')]
    df_filtered.iloc[:, 4:] = df_filtered.iloc[:, 4:].replace({'\u3000': ' '}, regex=True)
    prices = df_filtered.iloc[:, 3]
    buyers = df_filtered.iloc[:, 4:]
    buyer_data = defaultdict(lambda: {'count': 0, 'price': 0})

    for i, row in buyers.iterrows():
        price = int(prices[i])
        for buyer in row.dropna():
            buyer_data[buyer]['price'] += price
            buyer_data[buyer]['count'] += 1
    df_val = [[f"{key} ({val['count']})", 1, val['price']] for key, val in buyer_data.items()]
    df_buyer = pd.DataFrame(df_val, columns=['＊規格', '＊數量', '＊價格'])

    # Sort by selected option
    if sort_by == "筆畫":
        stroke_dict = load_stroke_data('kangxi-strokecount/kangxi-strokecount.csv')
        sorted_specifications = stroke_sort(df_buyer['＊規格'].tolist(), stroke_dict)

        # Reindex the DataFrame based on the sorted order
        df_buyer = df_buyer.set_index('＊規格').loc[sorted_specifications].reset_index()
    elif sort_by == "金額":
        df_buyer = df_buyer.sort_values(by="＊價格", ascending=False)
    elif sort_by == "不排序":
        pass
    return df_buyer


def select_files():
    file_paths = filedialog.askopenfilenames(title="選擇檔案", filetypes=[("Excel files", "*.xlsx *.xls")])
    if file_paths:
        file_listbox.delete(0, END)
        selected_files.clear()
        selected_files.extend(file_paths)

        for file in file_paths:
            file_listbox.insert(END, os.path.basename(file))


def display_buyer_data(sort_by=None):
    if not selected_files:
        showinfo("提示", "未選擇任何檔案")
        return

    df = read_data(selected_files)
    df_buyer = calculate_buyer_data(df, sort_by=sort_by)
    
    output_text.delete("1.0", END)
    output_text.insert(END, df_buyer.to_string(index=False))

# def save_batch_upload_file():
#     if not selected_files:
#         showinfo("提示", "未選擇任何檔案")
#         return

#     df = read_data(selected_files)
#     df_buyer = calculate_buyer_data(df)
    
#     save_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx *.xls")])
#     if save_path:
#         df_buyer.to_excel(save_path, index=False)
#         showinfo("成功", f"批次上架檔已儲存至 {save_path}")


def save_batch_upload_file():
    global last_sorting_method

    if last_sorting_method is None:
        showinfo("錯誤", "請先選擇排序方式！")
        return
    
    df = read_data(selected_files)
    df_buyer = calculate_buyer_data(df, sort_by=last_sorting_method)
    # Proceed with file generation using the last selected sorting method
    # Pass `last_sorting_method` to the generation function
    save_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx *.xls")])
    if save_path:
        df_buyer.to_excel(save_path, index=False)
        showinfo("成功", f"批次上架檔已儲存至 {save_path}")


def set_sorting_method(method):
    global last_sorting_method
    last_sorting_method = method
    display_buyer_data(sort_by=method)


def main_menu():
    for widget in root.winfo_children():
        widget.destroy()
    
    Label(root, text="請選擇功能", font=("Arial", menu_font_size)).pack(pady=20)
    Button(root, text="計算營收", font=("Arial", menu_font_size), command=revenue_menu).pack(pady=20)
    Button(root, text="批次上架", font=("Arial", menu_font_size), command=batch_upload_menu).pack(pady=20)


def adjust_window_size():
    root.update_idletasks()  # 更新視窗的子元件
    width = root.winfo_reqwidth()   # 獲取視窗需求的寬度
    height = root.winfo_reqheight() # 獲取視窗需求的高度
    root.geometry(f"{width}x{height}")  # 根據內容設置視窗大小


def revenue_menu():
    for widget in root.winfo_children():
        widget.destroy()
    
    Label(root, text="請選擇月份:", font=('Arial', submenu_font_size)).pack(pady=10)
    
    global month_listbox, file_listbox
    month_listbox = Listbox(root, selectmode='extended', width=20, height=12, exportselection=False, font=('Arial', submenu_font_size))
    months = [str(i) for i in range(1, 13)]
    for month in months:
        month_listbox.insert(END, month)
    month_listbox.pack()

    Button(root, text="選擇檔案", font=('Arial', submenu_font_size), command=select_files).pack(pady=10)
    Label(root, text="已選擇的檔案:", font=('Arial', submenu_font_size)).pack()
    file_listbox = Listbox(root, width=50, height=8, font=('Arial', submenu_font_size))
    file_listbox.pack()
    Button(root, text="開始計算", font=('Arial', submenu_font_size), command=calculate_revenue).pack(pady=20)
    Button(root, text="返回主選單", font=('Arial', submenu_font_size), command=main_menu).pack(pady=10)
    adjust_window_size()

def batch_upload_menu():
    global last_sorting_method  # Access the global variable

    for widget in root.winfo_children():
        widget.destroy()

    Label(root, text="批次上架工具", font=('Arial', submenu_font_size)).pack(pady=10)
    Button(root, text="選擇檔案", font=('Arial', submenu_font_size), command=select_files).pack(pady=5)
    Label(root, text="已選擇的檔案:", font=('Arial', submenu_font_size)).pack()
    
    global file_listbox
    file_listbox = Listbox(root, width=50, height=5, font=('Arial', submenu_font_size))
    file_listbox.pack()

    Button(root, text="不排序", font=('Arial', submenu_font_size), command=lambda: set_sorting_method("不排序")).pack(pady=5)
    Button(root, text="按買家筆畫排序", font=('Arial', submenu_font_size), command=lambda: set_sorting_method("筆畫")).pack(pady=5)
    Button(root, text="按金額排序", font=('Arial', submenu_font_size), command=lambda: set_sorting_method("金額")).pack(pady=5)

    global output_text
    output_text = Text(root, width=60, height=15)
    output_text.pack(pady=10)
    
    Button(root, text="生成批次上架檔", font=('Arial', submenu_font_size), command=save_batch_upload_file).pack(pady=10)
    Button(root, text="返回主選單", font=('Arial', submenu_font_size), command=main_menu).pack(pady=10)
    adjust_window_size()

root = Tk()
root.title("營收與買家統計工具")
selected_files = []

main_menu()

root.update_idletasks()
root.minsize(root.winfo_reqwidth(), root.winfo_reqheight())
root.geometry('300x200+100+100')
root.mainloop()


In [ ]:
from collections import defaultdict
import pandas as pd
from cn_sort.process_cn_word import *

df = pd.read_excel('../data/1108(14團)_P.xlsx')

def load_stroke_data(filepath):
    # Load CSV and create a dictionary of character to stroke count
    df = pd.read_csv(filepath, skiprows=4)
    stroke_dict = dict(zip(df['Character'], df['Strokes']))
    return stroke_dict

def stroke_sort(input_list, stroke_dict):
    def char_key(char):
        # If the character is in stroke dictionary, return its stroke count and char for further sorting
        if char in stroke_dict:
            return (stroke_dict[char], char)
        # Non-Chinese characters are placed with a default stroke count of 0 for initial sorting
        else:
            return (0, char)

    def string_key(s):
        # Convert each string into a tuple of sorting keys for each character
        return [char_key(char) for char in s]

    # Sort the list based on generated keys
    sorted_list = sorted(input_list, key=string_key)
    return sorted_list

# Example usage:

def calculate_buyer_data(df, sort_by=None):
    df_filtered = df.loc[df.iloc[:, 0].map(lambda x: x != '下架日/開售日')]
    df_filtered.iloc[:, 4:] = df_filtered.iloc[:, 4:].replace({'\u3000': ' '}, regex=True)
    prices = df_filtered.iloc[:, 3]
    buyers = df_filtered.iloc[:, 4:]
    buyer_data = defaultdict(lambda: {'count': 0, 'price': 0})

    for i, row in buyers.iterrows():
        price = int(prices[i])
        for buyer in row.dropna():
            buyer_data[buyer]['price'] += price
            buyer_data[buyer]['count'] += 1
    df_val = [[f"{key} ({val['count']})", 1, val['price']] for key, val in buyer_data.items()]
    df_buyer = pd.DataFrame(df_val, columns=['＊規格', '＊數量', '＊價格'])

    # Sort by selected option
    if sort_by == "筆畫":
        stroke_dict = load_stroke_data('kangxi-strokecount/kangxi-strokecount.csv')
        sorted_specifications = stroke_sort(df_buyer['＊規格'].tolist(), stroke_dict)

        # Reindex the DataFrame based on the sorted order
        df_buyer = df_buyer.set_index('＊規格').loc[sorted_specifications].reset_index()
    elif sort_by == "金額":
        df_buyer = df_buyer.sort_values(by="＊價格", ascending=False)

    return df_buyer

calculate_buyer_data(df, sort_by="筆畫")